In [ ]:
import pandas as pd
import h5py 
import numpy as np
import os,sys
import matplotlib.pyplot as plt
%matplotlib inline

Note: we could do this via GCRCatalogs, but I am just going to grab data from the hdf5 file directly.  For a quick and dirty example set, just grab data from the healpix 9816 with 10 year mock errors added, grab ~10,000 for training and ~20,000 for validation.

In [ ]:
pathx = "/global/cfs/cdirs/lsst/groups/PZ/PhotoZDC2/COSMODC2v1.1.4/IMAGE/10_year_error_estimates"
files = ['z_0_1.step_all.healpix_9816_magwerrSNtrim.hdf5','z_1_2.step_all.healpix_9816_magwerrSNtrim.hdf5','z_2_3.step_all.healpix_9816_magwerrSNtrim.hdf5']

In [ ]:
f = h5py.File(os.path.join(pathx,files[0]),'r')
p = f['photometry']
list(p.keys())

In [ ]:
for i,xfile in enumerate(files):
    f = h5py.File(os.path.join(pathx,xfile),'r')
    p = f['photometry']
    if i==0:
        pdict = {}
        for key in p.keys():
            pdict[f'{key}']=np.array(p[key])
    else:
        for key in p.keys():
            tmpy = np.array(p[key])
            pdict[f'{key}'] = np.concatenate([pdict[f'{key}'],tmpy])
    f.close()

In [ ]:
df = pd.DataFrame(pdict)
df.info()

In [ ]:
fig = plt.figure(figsize=(10,10))
plt.scatter(df['mag_i_lsst'][::100],df['redshift'][::100])

In [ ]:
mask = (df['mag_i_lsst']<25.3)
golddf = df[mask]

In [ ]:
golddf.info()

In [ ]:
traindf = golddf[::62]
traindf.info()

In [ ]:
tmpdf = golddf[1:]
validationdf = tmpdf[::31]

In [ ]:
validationdf.info()

In [ ]:
def create_output_file(outfile,xdf,num_rows):
    outf = h5py.File(outfile,"w")
    xx = outf.create_group('photometry')
    #outf.create_dataset('id', (num_rows,), dtype='int64')
    xx['id']=xdf['id'][:num_rows]
    for key in xdf.keys():
        print(key)
        if key != 'id':
            #outf.create_dataset(f'{key}', (num_rows,), dtype='f4')
            xx[f'{key}'] = xdf[key][:num_rows]
    outf.close()

In [ ]:
create_output_file("test_dc2_training_9816.hdf5",traindf,10_225)

In [ ]:
create_output_file("test_dc2_validation_9816.hdf5",validationdf,20_449)